In [116]:
import funct
import numpy as np
import pandas as pd

In [117]:
freq = 3
rela_path = "CCS_Ecuador_2024-03-07123224pm\CCS_Ecuador_2024-03-07123224pm.nc"

In [118]:
df = funct.create_time_series_dataframe(freq, rela_path)
df

,0,1,2,3,4,5,6,7,8,9,...,67309,67310,67311,67312,67313,67314,67315,67316,67317,67318
2024-03-05 00:00:00,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2024-03-05 03:00:00,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2024-03-05 06:00:00,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2024-03-05 09:00:00,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2024-03-05 12:00:00,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
